In [ ]:
import csv
from parsel.selector import Selector
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
import pandas as pd

In [ ]:
file = open('company_employee_details.csv', 'w+', encoding='utf-8-sig', newline='')
writer = csv.writer(file)
writer.writerow(['Name', 'Position', 'Company', 'Education', 'Location', 'URL'])

In [ ]:
driver = webdriver.Firefox()

In [ ]:
driver.get('https://linkedin.com/')

In [ ]:
try:
    sign_in_button = driver.find_element_by_class_name('nav__button-secondary')
    sign_in_button.click() if sign_in_button else None
    sleep(2)
except:
    pass

username = driver.find_element_by_name('session_key')
username.send_keys('robotnoa7@gmail.com')
sleep(3)

password = driver.find_element_by_name('session_password')
password.send_keys('mygalaxys9plus')
sleep(1)

try:
    sign_in_button = driver.find_element_by_class_name('sign-in-form__submit-button')
    sign_in_button.click() if sign_in_button else None
    sleep(0.5)
except:
    sign_in_button = driver.find_element_by_class_name('login__form_action_container')
    sign_in_button.click() if sign_in_button else None
    sleep(0.5)

In [ ]:
driver.get('https://www.google.com/')

In [ ]:
items = ['Boeing', 'Costco', 'Starbucks', 'Paccar', 'Wells Fargo', 'Zones LLC']

In [ ]:
for item in items:
    
    driver.get('https://www.google.com/')
    
    search_query = driver.find_element_by_name('q')
    search_query.send_keys('site:linkedin.com/in AND '+item)
    search_query.send_keys(Keys.RETURN)
    sleep(2)
    
    urls = driver.find_elements_by_xpath('//*[@class = "r"]/a[@href]')
    urls = [url.get_attribute('href') for url in urls]
    sleep(2.5)
    
    print('getting information from '+item+'............')
    
    for url in urls:
        driver.get(url)
        sleep(2)

        sel = Selector(text = driver.page_source)
        name = sel.xpath('//*[@class= "inline t-24 t-black t-normal break-words"]/text()').extract_first().split()
        name = ' '.join(name)
        
        sleep(2)

        try:
            experience = sel.xpath('//*[@class = "pv-profile-section__section-info section-info pv-profile-section__section-info--has-more"]')[0]
        except IndexError:
            try:
                experience = sel.xpath('//*[@class = "pv-profile-section__section-info section-info pv-profile-section__section-info--has-no-more"]')[0]
            except IndexError:
                experience = sel.xpath('//*[@class = "pv-profile-section__section-info section-info pv-profile-section__section-info--has-no-more"]')

        position = experience.xpath('//*[@class= "t-16 t-black t-bold"]/text()').extract_first()
        company = experience.xpath('//*[@class= "pv-entity__secondary-title t-14 t-black t-normal"]/text()').extract_first()
        
        position = ' '.join(position.split()) if position else None
        company = ' '.join(company.split()) if company else None
        
        if(company == None or position == None):
            company = experience.xpath('//*[@class = "t-16 t-black t-bold"]//span[2]/text()').extract_first()
            position = experience.xpath('//*[@class= "t-14 t-black t-bold"]//span[2]/text()').extract_first()
    
        if(company != None):
            flag = False
            for i in item.split():
                if(company.find(i) != -1):
                    flag = True
                    break
            if(flag==False):
                continue
            
        try:
            learnings = sel.xpath('//*[@class = "pv-profile-section__section-info section-info pv-profile-section__section-info--has-no-more"]/li')[-1]
        except IndexError:
            try:
                learnings = sel.xpath('//*[@class = "pv-profile-section__section-info section-info pv-profile-section__section-info--has-more"]/li')[-1]
            except:
                learnings = sel.xpath('//*[@class = "pv-profile-section__section-info section-info pv-profile-section__section-info--has-more"]/li')

        school = learnings.xpath('//*[@class = "pv-entity__school-name t-16 t-black t-bold"]/text()').extract_first()
        school = ' '.join(school.split()) if school else None

        des = learnings.xpath('//*[@class = "pv-entity__comma-item"]/text()').extract_first()
        des = ' '.join(des.split()) if des else None

        degree = school+": "+des if(type(school) == type(des) and des != None) else None

        location = sel.xpath('//*[@class= "t-16 t-black t-normal inline-block"]/text()').extract_first().split()
        location = ' '.join(location)

        url = driver.current_url

        writer.writerow([name, position, company, degree, location, url])

    print('parsed information from '+ item)
    print('')

In [ ]:
driver.quit()

In [ ]:
file.close()

In [ ]:
df = pd.read_csv('company_employee_details.csv')

In [ ]:
df.head()